In [88]:
import time
import csv
import numpy as np
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import warnings
from statistics import mean
import xgboost as xgb
from joblib import dump
from joblib import load

In [95]:
DataSet_="IPFIXRecords"
# Chargement des données d'entraînement du défenseur
X_train_defender = pd.read_csv('Dataset/'+DataSet_+'/X_train_defender.csv')
y_train_defender = pd.read_csv('Dataset/'+DataSet_+'/y_train_defender.csv')

# Chargement des données de test du défenseur
X_test_defender = pd.read_csv('Dataset/'+DataSet_+'/X_test_defender.csv')
y_test_defender = pd.read_csv('Dataset/'+DataSet_+'/y_test_defender.csv')

# Chargement des données de validation du défenseur
X_val_defender = pd.read_csv('Dataset/'+DataSet_+'/X_val_defender.csv')
y_val_defender = pd.read_csv('Dataset/'+DataSet_+'/y_val_defender.csv')

# Chargement des données d'entraînement de l'attaquant
X_train_attacker = pd.read_csv('Dataset/'+DataSet_+'/X_train_attacker.csv')
y_train_attacker = pd.read_csv('Dataset/'+DataSet_+'/y_train_attacker.csv')

# Chargement des données de test de l'attaquant
X_test_attacker = pd.read_csv('Dataset/'+DataSet_+'/X_test_attacker.csv')
y_test_attacker = pd.read_csv('Dataset/'+DataSet_+'/y_test_attacker.csv')

# Chargement des données de validation de l'attaquant
X_val_attacker = pd.read_csv('Dataset/'+DataSet_+'/X_val_attacker.csv')
y_val_attacker = pd.read_csv('Dataset/'+DataSet_+'/y_val_attacker.csv')


# Assurer que les formes sont correctes
print("Shape of X_train_attacker:", X_train_attacker.shape)
print("Shape of X_test_attacker:", X_test_attacker.shape)
print("Shape of y_val_attacker:", X_val_attacker.shape)
print("Shape of y_train_attacker:", y_train_attacker.shape)
print("Shape of y_test_attacker:", y_test_attacker.shape)
print("Shape of y_val_attacker:", y_val_attacker.shape)
print("Shape of X_train_defender:", X_train_defender.shape)
print("Shape of X_test_defender:", X_test_defender.shape)
print("Shape of y_train_defender:", X_val_defender.shape)
print("Shape of y_train_defender:", y_train_defender.shape)
print("Shape of y_test_defender:", y_test_defender.shape)
print("Shape of y_val_defender:", y_val_defender.shape)

y_train_encoded_attacker = to_categorical(y_train_attacker)
y_val_encoded_attacker = to_categorical(y_val_attacker)
y_test_encoded_attacker = to_categorical(y_test_attacker)
y_train_encoded_defender = to_categorical(y_train_defender)
y_val_encoded_defender = to_categorical(y_val_defender)
y_test_encoded_defender = to_categorical(y_test_defender)


scaler = StandardScaler()
X_train_scaled_attacker = scaler.fit_transform(X_train_attacker)
X_test_scaled_attacker = scaler.transform(X_test_attacker)
X_val_scaled_attacker = scaler.transform(X_val_attacker)

scaler = StandardScaler()
X_train_scaled_defender = scaler.fit_transform(X_train_defender)
X_test_scaled_defender = scaler.transform(X_test_defender)
X_val_scaled_defender = scaler.transform(X_val_defender)



Shape of X_train_attacker: (170542, 14)
Shape of X_test_attacker: (63165, 14)
Shape of y_val_attacker: (18950, 14)
Shape of y_train_attacker: (170542, 1)
Shape of y_test_attacker: (63165, 1)
Shape of y_val_attacker: (18950, 1)
Shape of X_train_defender: (170542, 14)
Shape of X_test_defender: (63164, 14)
Shape of y_train_defender: (18950, 14)
Shape of y_train_defender: (170542, 1)
Shape of y_test_defender: (63164, 1)
Shape of y_val_defender: (18950, 1)


In [96]:
warnings.filterwarnings("ignore", category=UserWarning)
# Sélection des paramètres
nmbVoisinsDefenseur = 3
nmbVoisinsAttaquant = 8
# Modèle KNN pour le défenseur
knn_defender = KNeighborsClassifier(n_neighbors=nmbVoisinsDefenseur)
knn_defender.fit(X_train_defender, y_train_defender)

# Prédiction et rapport de classification pour le défenseur
y_pred_defender = knn_defender.predict(X_test_defender)
report_defender = classification_report(y_test_defender, y_pred_defender, output_dict=True)
print("Perfermance de modèle de substitution KNN",report_defender['weighted avg'])
dump(knn_defender, 'Model\\' + DataSet_ + '\\DefenderModel\\knn_defender.joblib')

# Modèle KNN pour l'attaquant
knn_attacker = KNeighborsClassifier(n_neighbors=nmbVoisinsAttaquant)
knn_attacker.fit(X_train_attacker, y_train_attacker)

# Prédiction et rapport de classification pour l'attaquant
y_pred_attacker = knn_attacker.predict(X_test_attacker)
report_attacker = classification_report(y_test_attacker, y_pred_attacker, output_dict=True)
print("Perfermance de modèle de l'attaqaunt KNN",report_attacker['weighted avg'])

dump(knn_attacker, 'Model\\' + DataSet_ + '\\AttackerModel\\knn_attacker.joblib')

Perfermance de modèle de substitution KNN {'precision': 0.9725053304571101, 'recall': 0.9721835222595149, 'f1-score': 0.9719068198250235, 'support': 63164.0}
Perfermance de modèle de l'attaqaunt KNN {'precision': 0.9735210371383889, 'recall': 0.9729280455948706, 'f1-score': 0.9729333625994497, 'support': 63165.0}


['Model\\IPFIXRecords\\AttackerModel\\knn_attacker.joblib']

In [97]:
warnings.filterwarnings("ignore", category=UserWarning)

# RandomForest pour Défenseur
RandomForest_defender = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
RandomForest_defender.fit(X_train_defender, y_train_defender)
pred_defender = RandomForest_defender.predict(X_test_defender)
report_defender = classification_report(y_test_defender, pred_defender, output_dict=True)
print("Perfermance de modèle de substitution RF",report_defender['weighted avg'])


dump(RandomForest_defender, 'Model\\' + DataSet_ + '\\DefenderModel\\RandomForest_defender.joblib')

# RandomForest pour l'attaquant
RandomForest_attacker = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
RandomForest_attacker.fit(X_train_attacker, y_train_attacker)
pred_attacker = RandomForest_attacker.predict(X_test_attacker)
report_attacker = classification_report(y_test_attacker, pred_attacker, output_dict=True)
print("Perfermance de modèle de substitution RF",report_defender['weighted avg'])
dump(RandomForest_attacker, 'Model\\' + DataSet_ + '\\AttackerModel\\RandomForest_attacker.joblib')

Perfermance de modèle de substitution RF {'precision': 0.9769289052790292, 'recall': 0.9758565005382813, 'f1-score': 0.9759480510180886, 'support': 63164.0}
Perfermance de modèle de substitution RF {'precision': 0.9769289052790292, 'recall': 0.9758565005382813, 'f1-score': 0.9759480510180886, 'support': 63164.0}


['Model\\IPFIXRecords\\AttackerModel\\RandomForest_attacker.joblib']

In [98]:
warnings.filterwarnings("ignore", category=UserWarning)

input_shape_defender = X_train_defender.shape[1]
input_shape_attacker = X_train_attacker.shape[1]
nb_neurones_defender=256
nb_neurones_attacker=512

# DNN pour Défenseur
DNN_defender = keras.Sequential([
    keras.layers.Dense(nb_neurones_defender, activation='relu', input_shape=(input_shape_defender,)),
    keras.layers.Dense(nb_neurones_defender, activation='relu'),
    keras.layers.Dense(nb_neurones_defender, activation='relu'),
    keras.layers.Dense(18, activation='softmax')
])
DNN_defender.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],)
history_defender = DNN_defender.fit(X_train_scaled_defender, y_train_encoded_defender, epochs=10, batch_size=100)

y_pred_defender_prob = DNN_defender.predict(X_test_scaled_defender)
y_pred_defender = np.argmax(y_pred_defender_prob, axis=1)
report_defender = classification_report(y_test_defender, y_pred_defender,output_dict=True)
print("Perfermance de modèle de substitution KNN",report_defender['weighted avg'])

tf.keras.models.save_model(DNN_defender, 'Model\\' + DataSet_ + '\\DefenderModel\\DNN_defender.h5')

# DNN pour l'attaquant
DNN_attacker = keras.Sequential([
    keras.layers.Dense((nb_neurones_attacker), activation='relu', input_shape=(input_shape_attacker,)),
    keras.layers.Dense(nb_neurones_attacker, activation='relu'),
    keras.layers.Dense(nb_neurones_attacker, activation='relu'),
    keras.layers.Dense(18, activation='softmax')
])
DNN_attacker.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history_attacker = DNN_attacker.fit(X_train_scaled_attacker, y_train_encoded_attacker, epochs=10, batch_size=100, validation_data=(X_val_scaled_attacker, y_val_encoded_attacker))

y_pred_attacker_prob = DNN_attacker.predict(X_test_scaled_attacker)
y_pred_attacker= np.argmax(y_pred_attacker_prob, axis=1)
report_attacker = classification_report(y_test_attacker, y_pred_attacker,output_dict=True)
print("Perfermance de modèle de l'attaqaunt DNN",report_attacker['weighted avg'])

tf.keras.models.save_model(DNN_attacker, 'Model\\' + DataSet_ + '\\AttackerModel\\DNN_attacker.h5')

Epoch 1/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.6844 - loss: 0.8745
Epoch 2/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8510 - loss: 0.4358
Epoch 3/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8904 - loss: 0.3404
Epoch 4/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9042 - loss: 0.3041
Epoch 5/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9142 - loss: 0.2766
Epoch 6/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9223 - loss: 0.2450
Epoch 7/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9246 - loss: 0.2319
Epoch 8/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9238 - loss: 0.2272
Epoch 9/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9278 - loss: 0.2176
Epoch 10/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9285 - loss: 0.2092
1974/1974 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


Perfermance de modèle de substitution KNN {'precision': 0.9384728314117139, 'recall': 0.9295959723893357, 'f1-score': 0.9297126038877142, 'support': 63164.0}
Epoch 1/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.6910 - loss: 0.8298 - val_accuracy: 0.8358 - val_loss: 0.4339
Epoch 2/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8809 - loss: 0.3676 - val_accuracy: 0.8777 - val_loss: 0.3521
Epoch 3/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9044 - loss: 0.3024 - val_accuracy: 0.9268 - val_loss: 0.2500
Epoch 4/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9171 - loss: 0.2584 - val_accuracy: 0.9235 - val_loss: 0.2382
Epoch 5/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9171 - loss: 0.2520 - val_accuracy: 0.9281 - val_loss: 0.2120
Epoch 6/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9198 - loss: 0.2468 - val_accuracy: 0.8932 - val_loss: 0.2856
Epoch 7/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step -

Perfermance de modèle de l'attaqaunt DNN {'precision': 0.9375244203813856, 'recall': 0.9324625979577298, 'f1-score': 0.9321216709200603, 'support': 63165.0}


In [99]:
warnings.filterwarnings("ignore", category=UserWarning)

#XGBoost pour Défenseur
# Définir les paramètres du modèle XGBoost
params_defender = {
    'objective': 'multi:softmax',  # objectif de classification multiclasse
    'num_class': 18,  # nombre de classes
    'eval_metric': 'merror'  # métrique d'évaluation (taux d'erreur de classification)
}
params_attacker  = {
    'objective': 'multi:softmax',  
    'num_class': 18,  
    'eval_metric': 'merror' 
}

# Convertir les données au format DMatrix pour XGBoost
dtrain_defender = xgb.DMatrix(X_train_defender, label=y_train_defender)
dval_defender = xgb.DMatrix(X_val_defender, label=y_val_defender)
dtest_defender = xgb.DMatrix(X_test_defender)

dtrain_attacker = xgb.DMatrix(X_train_attacker, label=y_train_attacker)
dval_attacker = xgb.DMatrix(X_val_attacker, label=y_val_attacker)
dtest_attacker = xgb.DMatrix(X_test_attacker)

n_Estimateur_attaquant = 100 #nombre d'estimateur
n_Estimateur_defender = 200 
XGBoost_defender = xgb.train(params=params_defender,dtrain=dtrain_defender,num_boost_round=n_Estimateur_defender,)
pred_defender = XGBoost_defender.predict(dtest_defender)

report_defender = classification_report(y_test_defender, pred_defender, output_dict=True)
print("Perfermance de modèle de substitution XGBoost",report_defender['weighted avg'])
dump(XGBoost_defender, 'Model\\' + DataSet_ + '\\DefenderModel\\XGBoost_defender.joblib')



#XGBoost pour l'attaquant
XGBoost_attacker = xgb.train(params=params_attacker ,dtrain=dtrain_attacker,num_boost_round=n_Estimateur_attaquant,)
pred_attacker = XGBoost_attacker.predict(dtest_attacker)
report_attacker = classification_report(y_test_attacker, pred_attacker, output_dict=True)
print("Perfermance de modèle de l'attaqaunt XGBoost",report_attacker['weighted avg'])

dump(XGBoost_attacker, 'Model\\' + DataSet_ + '\\AttackerModel\\XGBoost_attacker.joblib')


Perfermance de modèle de substitution XGBoost {'precision': 0.9765901932283909, 'recall': 0.9750807421949211, 'f1-score': 0.975248813348119, 'support': 63164.0}
Perfermance de modèle de l'attaqaunt XGBoost {'precision': 0.9769692454035517, 'recall': 0.9763318293358664, 'f1-score': 0.9759738646613769, 'support': 63165.0}


['Model\\IPFIXRecords\\AttackerModel\\XGBoost_attacker.joblib']

In [100]:
warnings.filterwarnings("ignore", category=UserWarning)

fct_qualite_defender="entropy"
fct_qualite_attacker="gini"
DecisionTree_defender = DecisionTreeClassifier(criterion=fct_qualite_defender, random_state=0)
DecisionTree_defender.fit(X_train_defender, y_train_defender)
pred_defender = DecisionTree_defender.predict(X_test_defender)
report_defender = classification_report(y_test_defender, pred_defender, output_dict=True)
print("Perfermance de modèle de substitution DT",report_defender['weighted avg'])
dump(DecisionTree_defender, 'Model\\' + DataSet_ + '\\DefenderModel\\DecisionTree_defender.joblib')


DecisionTree_attacker = DecisionTreeClassifier(criterion=fct_qualite_attacker, random_state=0)
DecisionTree_attacker.fit(X_train_attacker, y_train_attacker)
pred_attacker = DecisionTree_attacker.predict(X_test_attacker)
report_attacker = classification_report(y_test_attacker, pred_attacker, output_dict=True)
print("Perfermance de modèle de l'attaqaunt DT",report_attacker['weighted avg'])
dump(DecisionTree_attacker, 'Model/IPFIXHome/AttackerModel/DecisionTree_attacker.joblib')


Perfermance de modèle de substitution DT {'precision': 0.9758887352761707, 'recall': 0.9751757330124755, 'f1-score': 0.9751653005530446, 'support': 63164.0}
Perfermance de modèle de l'attaqaunt DT {'precision': 0.9763667865776678, 'recall': 0.976031029842476, 'f1-score': 0.9757026243692485, 'support': 63165.0}


['Model/IPFIXHome/AttackerModel/DecisionTree_attacker.joblib']